In [1]:
import os

In [2]:
%pwd

'c:\\Users\\GMI-00108\\Downloads\\Data Science Project(CICD)\\Data-Science-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd       # i have changed my working directory to this because i ingest the data by creating artifacts folder in that we will import our data
#SO, Artifacts folder should be created in this directory. We can create our structure and import dataset in Config.yml file

'c:\\Users\\GMI-00108\\Downloads\\Data Science Project(CICD)\\Data-Science-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path
  
@dataclass                # No need of initiazling __init__
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_directory: Path


In [11]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self,config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_directory = config.unzip_directory
        )
        return data_ingestion_config


In [7]:
%pwd

'c:\\Users\\GMI-00108\\Downloads\\Data Science Project(CICD)\\Data-Science-Project'

In [ ]:
from src.datascience import logger
from urllib import request
import zipfile
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"file already exists")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_directory
        os.makedirs(unzip_path,exist_ok = True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [16]:
try:
    config = ConfigurationManager()
    get_data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = get_data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-10 10:38:00,853: INFO: common: yaml file: config\config.yaml file loaded successfully]
[2025-02-10 10:38:00,855: INFO: common: yaml file: params.yaml file loaded successfully]
[2025-02-10 10:38:00,857: INFO: common: yaml file: schema.yaml file loaded successfully]
[2025-02-10 10:38:00,858: INFO: common: created directory at artifacts/data_ingestion]
[2025-02-10 10:38:02,356: INFO: 2658734969: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1EDB:25CC20:43E4D3:650478:67A989B4
Accept-Ranges: bytes
Date: Mon, 10 Feb 2025 05:08:04 GMT
Via: 1.1 varnish
X-Served-By: cache-

In [ ]:
from pathlib import Path

PARAMS_FILE_PATH = Path("params.yaml")
print(PARAMS_FILE_PATH.exists()) 

True


In [ ]:
# config/config.yaml
import os
from dataclasses import dataclass
from pathlib import Path
import urllib.request
import zipfile
from src.datascience import logger
from src.datascience.utils.common import read_yaml, create_directories

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_directory: Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        # Convert string paths to Path objects if they aren't already
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        # Create root directory
        create_directories([config.root_dir])
        
        # Convert string paths to Path objects
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_directory=Path(config.unzip_directory)
        )
        return data_ingestion_config

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Create parent directories if they don't exist
            os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
            
            try:
                filename, headers = urllib.request.urlretrieve(
                    url=self.config.source_URL,
                    filename=str(self.config.local_data_file)  # Convert Path to string
                )
                logger.info(f"File downloaded: {filename} with headers: \n{headers}")
            except Exception as e:
                logger.error(f"Error downloading file: {str(e)}")
                raise e
        else:
            logger.info(f"File already exists at: {self.config.local_data_file}")
    
    def extract_zip_file(self):
        try:
            unzip_path = self.config.unzip_directory
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(str(self.config.local_data_file), 'r') as zip_ref:
                zip_ref.extractall(str(unzip_path))
            logger.info(f"Files extracted to: {unzip_path}")
        except Exception as e:
            logger.error(f"Error extracting zip file: {str(e)}")
            raise e

# Usage
def main():
    try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)  # Pass the config
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        logger.error(f"Error in data ingestion process: {str(e)}")
        raise e

if __name__ == "__main__":
    main()

[2025-02-10 10:27:04,348: INFO: common: yaml file: config\config.yaml file loaded successfully]
[2025-02-10 10:27:04,350: INFO: common: yaml file: params.yaml file loaded successfully]
[2025-02-10 10:27:04,351: ERROR: 1929590501: Error in data ingestion process: yaml file is empty]


ValueError: yaml file is empty